In [ ]:
import os
import tabulate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rmgpy.tools.uncertainty import Uncertainty, process_local_results
from rmgpy.tools.canteramodel import get_rmg_species_from_user_species
from rmgpy.species import Species
from rmgpy.tools.plot import parse_csv_data


# these are for showing the flux diagram
from IPython.display import Image


In [ ]:
# Load the model
# Must use annotated chemkin file

model_dir = '/home/moon/rmg/rmg_tools/uncertainty/nheptane'


chemkin_file = os.path.join(model_dir, 'chem_annotated.inp')
dict_file = os.path.join(model_dir, 'species_dictionary.txt')

# Initialize the Uncertainty class instance and load the model
uncertainty = Uncertainty(output_directory=os.path.join(model_dir, 'chemkin'))
uncertainty.load_model(chemkin_file, dict_file)


In [ ]:
# optional - generate flux diagram:

# from rmgpy.tools import fluxdiagram

# input_file = os.path.join(model_dir, 'input.py')
# output_path = os.path.join(model_dir, 'flux_diagram')
# fluxdiagram.create_flux_diagram(input_file, chemkin_file, dict_file, save_path=output_path)

Ethane pyrolysis produces ethylene (C2H4)

In [ ]:
# Image(filename="/home/moon/rmg/my_examples/ethane/flux_diagram/1/flux_diagram_0148.png")


In [ ]:
# for sp in uncertainty.species_list:
#     print(sp)
uncertainty.species_list[4].smiles

In [ ]:
# Map the species to the objects within the Uncertainty class
nheptane = Species().from_smiles('CCCCCCC')
CO2 = Species().from_smiles('O=C=O')
O2 = Species().from_smiles('[O][O]')
OH_rad = Species().from_smiles('[OH]')
Ne = Species().from_smiles('[Ne]')
mapping = get_rmg_species_from_user_species([nheptane, CO2, O2, OH_rad, Ne], uncertainty.species_list)


# TODO get model with He instead of Ne
# equivalence ratio phi, 11 O2 per n-heptane in stoichiometric feed
x_nheptane = 0.005
phi = 1.0
x_O2 = 11.0 * x_nheptane / phi
x_Ne = 1.0 - x_nheptane - x_O2
# print(x_nheptane)
# print(x_O2)
# print(x_Ne)

# Reaction conditions to match at least some Zhang(2016) experimental data
initial_mole_fractions = {
    mapping[nheptane]: x_nheptane,
    mapping[O2]: x_O2,
    mapping[Ne]: x_Ne,
}
T = (1000, 'K')
P_torr = 800.0
P_atm = P_torr / 760.0
P_Pa = P_atm * 101325.0
P_bar = P_Pa / 100000.0

P = (P_bar, 'bar')
termination_time = (0.5, 'ms')
sensitive_species = [mapping[nheptane], mapping[CO2], mapping[O2], mapping[OH_rad]]


In [ ]:
# Perform the sensitivity analysis
uncertainty.sensitivity_analysis(initial_mole_fractions, sensitive_species, T, P, termination_time, number=5, fileformat='.png')

In [ ]:
# Show the sensitivity plots
for species in sensitive_species:
    print('{}: Reaction Sensitivities'.format(species))
    index = species.index
    display(Image(filename=os.path.join(uncertainty.output_directory,'solver','sensitivity_1_SPC_{}_reactions.png'.format(index))))
    
    print('{}: Thermo Sensitivities'.format(species))
    display(Image(filename=os.path.join(uncertainty.output_directory,'solver','sensitivity_1_SPC_{}_thermo.png'.format(index))))

In [ ]:
# NOTE: You must load the database with the same settings which were used to generate the model.
#       This includes any thermo or kinetics libraries which were used.
uncertainty.load_database(
    thermo_libraries=[
        'BurkeH2O2',
        'CurranPentane',
        'FFCM1(-)',
        'primaryThermoLibrary',
        'thermo_DFT_CCSDTF12_BAC',
        'DFT_QCI_thermo',
        'CBS_QB3_1dHR'
    ],
    kinetics_families='default',
    reaction_libraries=[
        'CurranPentane',
        'FFCM1(-)',
        'combustion_core/version5'
    ]
)

In [ ]:
uncertainty.extract_sources_from_model()
uncertainty.assign_parameter_uncertainties()

In [ ]:
result = uncertainty.local_analysis(sensitive_species, correlated=False, number=15, fileformat='.png')
print(process_local_results(result, sensitive_species, number=15)[1])

In [ ]:
uncertainty.species_sources_dict

In [ ]:
def get_species_index(list_of_species, name):
    for i, sp in enumerate(list_of_species):
        if str(sp) == name:
            return i
    return -1


In [ ]:
C5H11_1_name = 'C5H11-1(422)'
C5H11_1 = uncertainty.species_list[get_species_index(uncertainty.species_list, C5H11_1_name)]

In [ ]:
uncertainty.species_sources_dict[C5H11_1]

In [ ]:
top_uncertain = []
names = [
    'nheptane(1)',
    'C5H11-1(422)',
    'PC4H9(184)',
    'NC3H7(93)',
    'H(14)',
    'C2H4(11)',
    'C2H5(44)',
    'C=CCCCC(810)',
    'CH3(19)',
    'CCC=CCCC(809)',
    'CC=CCCCC(755)',
    'CC[CH]CCCC(680)',
    'O2(2)',
    'C3H6(12)',
    'CH2(24)',
]
for name in names:
#     print(name)
    i = get_species_index(uncertainty.species_list, name)
    if i < 0:
        print(name)
    top_uncertain.append(uncertainty.species_list[i])

# print(top_uncertain)
# print(len(top_uncertain), len(names))

In [ ]:
for sp in top_uncertain:
    if 'GAV' in uncertainty.species_sources_dict[sp]:
        print(sp)

In [ ]:
for sp in uncertainty.species_list:
    print(str(sp))

# Reactions

In [ ]:
# k200: C2H5(44)+C5H11(422)=C7H16(1)                                      35.4632%
# k210: NC3H7(93)+PC4H9(184)=C7H16(1)                                     34.7859%
# k290: O(5)+C7H16(1)=OH(15)+C7H15(684)                                    8.1577%
# k252: H(14)+C7H16(1)=H2(13)+C7H15(680)                                   3.3216%
# k236: H(14)+C7H16(1)=H2(13)+C7H15(683)                                   2.6024%
# k272: H(14)+C7H16(1)=H2(13)+C7H15(681)                                   0.4817%
# k1: O2(2)+H(14)=O(5)+OH(15)                                              0.3560%
# k38: O(5)+C2H4(11)=CH2(24)+CH2O(9)                                       0.3357%
# k189: H(14)+C2H4(11)(+M)=C2H5(44)(+M)                                    0.1974%
# k194: O2(2)+C2H5(44)=HO2(17)+C2H4(11)                                    0.1548%
# k705: CH3(19)+C6H12(810)=C7H15(680)                                      0.1437%
# k55: O2(2)+CH2(24)=OH(15)+HCO(20)                                        0.0963%
# k259: C7H15(680)=C7H15(683)                                              0.0884%
# k237: CH3(19)+C7H16(1)=CH4(10)+C7H15(683)                                0.0802%
# k102: O2(2)+CH2(24)=>H(14)+H(14)+CO2(7)                                  0.0669%
top_ks = [199, 209, 289, 251, 235, 271, 0, 37, 188, 193, 704, 54, 258, 236, 101]
for i in top_ks:
    rxn = uncertainty.reaction_list[i]
    print(uncertainty.reaction_sources_dict[rxn])
#     print(uncertainty.reaction_sources_dict[rxn].keys())
#     if 'Rate Rules' in uncertainty.reaction_sources_dict[rxn]:
#         print(i, rxn)

In [ ]:
uncertainty.reaction_sources_dict[uncertainty.reaction_list[199]]

In [ ]:
uncertainty

In [ ]:
reaction_system_index = 0
sens_species = sensitive_species[1]

csvfile_path = os.path.join(
    uncertainty.output_directory,
    'solver',
    'sensitivity_{0}_SPC_{1}.csv'.format(
        reaction_system_index + 1,
        sens_species.index)
)
time, data_list = parse_csv_data(csvfile_path)

In [ ]:
time

In [ ]:
spec_names = []
for sp in uncertainty.species_list:
    spec_names.append(str(sp))
    
rxn_names = []
for rxn in uncertainty.reaction_list:
    rxn_names.append(str(rxn))

In [ ]:
thermo_data = []
kinetics_data = []
combined_data = []
for i, data in enumerate(data_list):
    if data.species:
        for j, sp in enumerate(uncertainty.species_list):
            if sp.to_chemkin() == data.species:
                product = np.abs(data.data[-1]) * uncertainty.thermo_input_uncertainties[j]
                thermo_data.append([j, spec_names[j], np.abs(data.data[-1]), uncertainty.thermo_input_uncertainties[j], product])
                combined_data.append([j, spec_names[j], np.abs(data.data[-1]), uncertainty.thermo_input_uncertainties[j], product])
                break
    if data.reaction:
        k = data.index - 1
        product = np.abs(data.data[-1]) * uncertainty.kinetic_input_uncertainties[k]
        kinetics_data.append([k, rxn_names[k], np.abs(data.data[-1]), uncertainty.kinetic_input_uncertainties[k], product])
        combined_data.append([k, rxn_names[k], np.abs(data.data[-1]), uncertainty.kinetic_input_uncertainties[k], product])
    


In [ ]:
thermo_df = pd.DataFrame(thermo_data, columns=['i', 'name', 'sensitivity', 'uncertainty', 'products'])
thermo_df.sort_values('sensitivity', ascending=False, inplace = True)

kinetics_df = pd.DataFrame(kinetics_data, columns=['i', 'name', 'sensitivity', 'uncertainty', 'products'])
kinetics_df.sort_values('sensitivity', ascending=False, inplace = True)

combined_df = pd.DataFrame(combined_data, columns=['i', 'name', 'sensitivity', 'uncertainty', 'products'])
combined_df.sort_values('sensitivity', ascending=False, inplace = True)

In [ ]:
%matplotlib inline
N = 20

plt.bar(thermo_df.name[0:N], thermo_df.sensitivity[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:
%matplotlib inline
N = 20

plt.bar(kinetics_df.name[0:N], kinetics_df.sensitivity[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:
%matplotlib inline
N = 20

plt.bar(combined_df.name[0:N], combined_df.sensitivity[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:
thermo_df.sort_values('uncertainty', ascending=False, inplace = True)
%matplotlib inline
N = 20

plt.bar(thermo_df.name[0:N], thermo_df.uncertainty[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:

kinetics_df.sort_values('uncertainty', ascending=False, inplace = True)
%matplotlib inline
N = 20

plt.bar(kinetics_df.name[0:N], kinetics_df.uncertainty[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:
combined_df.sort_values('uncertainty', ascending=False, inplace = True)
%matplotlib inline
N = 20

plt.bar(combined_df.name[0:N], combined_df.uncertainty[0:N])
plt.xticks(rotation=90)
plt.show()

In [ ]:
thermo_df.sort_values('products', ascending=False, inplace=True)
%matplotlib inline
N = 20

plt.figure(figsize=(14, 5), dpi=80)
plt.subplot(121)
plt.bar(thermo_df.name[0:N], thermo_df.uncertainty[0:N], label='uncertainty', bottom=thermo_df.sensitivity[0:N])
plt.bar(thermo_df.name[0:N], thermo_df.sensitivity[0:N], label='sensitivity')
plt.xticks(rotation=90)
plt.legend()

plt.subplot(122)
ax = plt.gca()

plt.bar(thermo_df.name[0:N], thermo_df.products[0:N], label='products', color=plt.rcParams['axes.prop_cycle'].by_key()['color'][2])
plt.xticks(rotation=90)
plt.legend()
plt.show()


In [ ]:
thermo_df.sort_values('products', ascending=False, inplace=True)
%matplotlib inline
N = 20

# plt.figure(figsize=(14, 5), dpi=80)
# plt.subplot(121)
base = 100
log_sense = np.log(100 * thermo_df.sensitivity.values[0:N]) / np.log(base)

plt.bar(thermo_df.name[0:N], np.log(100 * thermo_df.uncertainty.values[0:N]) / np.log(base), label='uncertainty', bottom=log_sense)
plt.bar(thermo_df.name[0:N], log_sense, label='sensitivity')



# plt.bar(thermo_df.name[0:N], thermo_df.products[0:N], label='products', color=plt.rcParams['axes.prop_cycle'].by_key()['color'][2])
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
kinetics_df.sort_values('products', ascending=False, inplace=True)
%matplotlib inline
N = 20


base = 100
log_sense_k = np.log(100 * kinetics_df.sensitivity.values[0:N]) / np.log(base)

plt.bar(kinetics_df.name[0:N], np.log(100 * kinetics_df.uncertainty.values[0:N]) / np.log(base), label='uncertainty', bottom=log_sense_k)
plt.bar(kinetics_df.name[0:N], log_sense_k, label='sensitivity')



# plt.bar(thermo_df.name[0:N], thermo_df.products[0:N], label='products', color=plt.rcParams['axes.prop_cycle'].by_key()['color'][2])
plt.xticks(rotation=90)
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
plt.plot(time.data, data_list[2].data)
plt.show()

In [ ]:
print(data_list[2].units)